In [1]:
pip install nltk -U

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pyspark

sc = spark.sparkContext

print('Original spark.driver.maxResultSize: ' + sc._conf.get('spark.driver.maxResultSize'))
sc.stop()

Original spark.driver.maxResultSize: 1920m


# PAUSE for sc to stop

In [3]:
conf = pyspark.SparkConf().setAll([('spark.driver.maxResultSize','4g')])
sc = pyspark.SparkContext(conf=conf)
print('New spark.driver.maxResultSize: ' + sc._conf.get('spark.driver.maxResultSize'))
spark = SparkSession.builder.config(conf=conf).getOrCreate()

22/12/07 03:42:55 INFO org.apache.spark.SparkEnv: Registering MapOutputTracker
22/12/07 03:42:55 INFO org.apache.spark.SparkEnv: Registering BlockManagerMaster
22/12/07 03:42:55 INFO org.apache.spark.SparkEnv: Registering BlockManagerMasterHeartbeat
22/12/07 03:42:55 INFO org.apache.spark.SparkEnv: Registering OutputCommitCoordinator


New spark.driver.maxResultSize: 4g


In [4]:
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows',None)
from itertools import compress 
from pyspark.sql.types import *
from pyspark.sql import functions as F
import seaborn as sns
import matplotlib.pyplot as plt
warnings.filterwarnings(action='ignore')

In [5]:
import re
from pyspark.ml.feature import MinHashLSH
from pyspark.ml.feature import CountVectorizer,  IDF, CountVectorizerModel, Tokenizer, RegexTokenizer, StopWordsRemover
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql import Row
from pyspark.sql import Window
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
spark.conf.set("spark.sql.repl.eagerEval.enabled",True)

In [7]:
import os
import subprocess
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [8]:
directory = 'gs://msca-bdp-tweets/final_project/'
path = directory

cmd = 'hadoop fs -du -s -h ' + directory

p = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, universal_newlines=True)
for line in p.stdout.readlines():
    print (line)
    
retval = p.wait()

497.0 G  497.0 G  gs://msca-bdp-tweets/final_project



In [9]:
tweets_df = spark.read.json(path)

22/12/07 03:44:00 WARN org.apache.spark.sql.execution.datasources.SharedInMemoryCache: Evicting cached table partition metadata from memory due to size constraints (spark.sql.hive.filesourcePartitionFileCacheSize = 262144000 bytes). This may impact query planning performance.
22/12/07 03:55:21 WARN org.apache.spark.sql.catalyst.util.package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [41]:
print((tweets_df.count(), len(tweets_df.columns)))

22/12/07 06:02:09 WARN org.apache.spark.deploy.yarn.YarnAllocator: Container from a bad node: container_1670264640795_0033_01_000089 on host: hub-msca-bdp-dphub-students-chw387-sw-zcxx.c.msca-bdp-students.internal. Exit status: 143. Diagnostics: [2022-12-07 06:02:09.759]Container killed on request. Exit code is 143
[2022-12-07 06:02:09.764]Container exited with a non-zero exit code 143. 
[2022-12-07 06:02:09.767]Killed by external signal
.
22/12/07 06:02:09 WARN org.apache.spark.deploy.yarn.YarnAllocator: Container from a bad node: container_1670264640795_0033_01_000091 on host: hub-msca-bdp-dphub-students-chw387-sw-zcxx.c.msca-bdp-students.internal. Exit status: 143. Diagnostics: [2022-12-07 06:02:09.764]Container killed on request. Exit code is 143
[2022-12-07 06:02:09.768]Container exited with a non-zero exit code 143. 
[2022-12-07 06:02:09.768]Killed by external signal
.
22/12/07 06:02:09 WARN org.apache.spark.scheduler.cluster.YarnSchedulerBackend$YarnSchedulerEndpoint: Requesting

(99992797, 40)


in_reply_to_status_id_str
in_reply_to_user_id_str
quoted_status_id_str
quoted_status

In [51]:
tweets_df.select(
    F.lower(F.col('tweet_text')).alias('lower_tweet_txt')
).filter(
    F.col('lower_tweet_txt').contains('primary school') &
    (~F.col('lower_tweet_txt').contains('shooting'))
).sample(0.01).limit(10).toPandas()

,lower_tweet_txt
0,"aussie primary school sees ""microchips in student's brains"" in 10 years https://t.co/yyonupffj5"
1,state-funded primary schools across the united kingdom will be sent a free commemorative book in may to mark queen elizabeth's platinum jubilee\nnow nicola sturgeon make sure scotlands children get their free book they all should have been presented with their free book in may.
2,the drama way i acted as a poor man for primary school nu ebi like say god carry am serious ☹️
3,these books helped us a lot in primary school 😭😭 https://t.co/maj8bg9yow
4,fantastic visit today to moulsecoomb primary school @moulsecoombsch. great to see happy children enjoying learning - the result of lots of hard work by school staff and the pioneer academy trust @thepioneertpa https://t.co/rypt4etm3b
5,"“you’re a b!tch” \n\ngirls in primary school : “ yes i am a b!itch , a b!tch is a female dog . dogs bark and bark grows on trees . trees are nature and nature is beautiful so that means i’m beautiful 🙄”"
6,smoke breaks are just an evolved practice of 5 kids sharpening their pencils around the bin in primary school
7,"@profshekharrjd\n\n#primary_notification_2022 \n\naccording to rte act criteria,which stipulates a pupil-teacher ratio–the number of pupils per teacher–of 30:1in primary schools &amp;35:1in uper primary https://t.co/b3p3szatln bihar more than 2lacks seats vacant?\n\n@dipakkrias"
8,"voted, met a divine ""democracy sausage"" called eddie outside the polling booth, and got an apple cake from the local primary school's stall. winning!"
9,@blkjjk1 14 kids and 1 teacher from a primary school died because of some 18yo moron from what i've seen. (they also shot their grandma if that wasn't enough)


In [ ]:
# Below just finished running, need to basically rerun entire filtered tweets pipeline
# remember to save a intermediary step 

In [10]:
filtered_tweets = tweets_df.select([
        'tweet_text','created_at','user','retweeted_status','place',
        lower(F.col('tweet_text')).alias('lower_tweet_txt'),F.col('id_str').alias('tweet_id')
    ]).filter(
        (
            F.col('possibly_sensitive').isNull
        ) & (
            ~F.col('lower_tweet_txt').contains('shoot') &
            ~F.col('lower_tweet_txt').contains('shot') &
            ~F.col('lower_tweet_txt').contains('kill') &
            ~F.col('lower_tweet_txt').contains('prayer')&
            ~F.col('lower_tweet_txt').contains('gun')&
            ~F.col('lower_tweet_txt').contains('championship')&
            ~F.col('lower_tweet_txt').contains('columbine')&
            ~F.col('lower_tweet_txt').contains('attack')&
            ~F.col('lower_tweet_txt').contains('murder')&
            ~F.col('lower_tweet_txt').contains('massacre')&
            ~F.col('lower_tweet_txt').contains('horny')&
            ~F.col('lower_tweet_txt').contains('scored')&
            ~F.col('lower_tweet_txt').contains('electoral college')&
            ~F.col('lower_tweet_txt').contains('coach')&
            ~F.col('lower_tweet_txt').contains('offseason')&
            ~F.col('lower_tweet_txt').contains('vaxxed')&
            ~F.col('lower_tweet_txt').contains('team')
        ) & (
            F.col('lower_tweet_txt').contains('grade') |
            F.col('lower_tweet_txt').contains('kindergarten')|
            F.col('lower_tweet_txt').contains('primary school') |
            F.col('lower_tweet_txt').contains('secondary school') |
            F.col('lower_tweet_txt').contains('education') |
            F.col('lower_tweet_txt').contains('tuition')|
            F.col('lower_tweet_txt').contains('undergrad')|
            F.col('lower_tweet_txt').contains('graduate')|
            F.col('lower_tweet_txt').contains('literacy')|
            F.col('lower_tweet_txt').contains('teacher')|
            F.col('lower_tweet_txt').contains('curriculum')|
            F.col('lower_tweet_txt').contains('learning')|
            F.col('lower_tweet_txt').contains('middle school')|
            F.col('lower_tweet_txt').contains('high school')|
            F.col('lower_tweet_txt').contains('elementary school')|
            F.col('lower_tweet_txt').contains('stem')|
            F.col('lower_tweet_txt').contains('math')|
            F.col('lower_tweet_txt').contains('humanities')|
            F.col('lower_tweet_txt').contains('phd')|
            F.col('lower_tweet_txt').contains('student')|
            F.col('lower_tweet_txt').contains('study')|
            F.col('lower_tweet_txt').contains('book')
        ) & (~(
            F.col('lower_tweet_txt').contains('vs') & 
            F.col('lower_tweet_txt').contains('stream')
        )) & (~(
            F.col('lower_tweet_txt').contains('football') |
            F.col('lower_tweet_txt').contains('basketball') |
            F.col('lower_tweet_txt').contains('watch live') |
            F.col('lower_tweet_txt').contains('soccer') |
            F.col('lower_tweet_txt').contains('𝑻𝑶𝑫𝑨𝒀❜𝑺 𝑴𝑨𝑻𝑪𝑯')       
        )) & (~(
            (
                F.col('lower_tweet_txt').contains('watch') |
                F.col('lower_tweet_txt').contains('𝐖𝐀𝐓𝐂𝐇')
            ) & (
                F.col('lower_tweet_txt').contains('stream') |
                F.col('lower_tweet_txt').contains('𝐒𝐓𝐑𝐄𝐀𝐌') |
                F.col('lower_tweet_txt').contains('live')|
                F.col('lower_tweet_txt').contains('𝐋𝐈𝐕𝐄')
            )
        ))
)

In [ ]:
filtered_tweets.count()

29970964

In [76]:
print('contains tweets from {} to {}'.format(
    filtered_tweets.agg(F.min('created_at')).collect()[0][0].strftime('%Y-%m-%d'),
    filtered_tweets.agg(F.max('created_at')).collect()[0][0].strftime('%Y-%m-%d')
))

contains tweets from 2022-04-05 to 2022-11-06


double check selected words work

In [35]:
(
    filtered_tweets
    .select('lower_tweet_txt')
    .filter(F.col('lower_tweet_txt').contains('book'))
).sample(fraction=0.2).limit(20).toPandas()

,lower_tweet_txt
0,"christian and muslim parents have joined forces to protest a dearborn public schools meeting in michigan to protest the inclusion of ""woke"" lgbtq+ books in schools that contain sexually explicit material. https://t.co/5g0lz9os8o"
1,"@anandwrites @gregabbott_tx @persuadersbook i went to high school with anand....he used to pick fights with girls there, too."
2,my addiction to buying things i don’t need started at the school book fair
3,aimim party president barrister asaduddin owaisi today distributes bags and note books all in one's to urdu medium school students at party headquarters darussalam. @asadowaisi https://t.co/rogghqyit0
4,"we gave 50 children without school bags new cool back packs with head phones, chess sets and harry potter books for the new school session. https://t.co/zltctzulm4"
5,"delighted to share the summary of the brilliant how teaching happens by @p_a_kirschner, \n@c_hendrick, @healjim (approved by paul!)- every school should have a copy of the book!\n\npdf is here + the summary of how learning happens - also excellent). \nhttps://t.co/jdxge3xleu https://t.co/umxizasnw3"
6,hindi was one of my most favourite subject in school &amp; i feel lucky to be able to read stories by parsai &amp; premchand in textbooks and at the same time getting to watch tv serials based on their work back then. still regret that i couldn't continue that in college \n\n+ https://t.co/9xakm4lki8
7,"@dpradhanbjp sir, pl. look at our site, in corona two years we paid full fees to school, no discount, and now school increase fees, and open business for textbook, uniform, shoes, why ? school are sale all products then vocal for local discontinue from all india"
8,"school make syndicate their favorite vendor, then other vendor where go to sale? our school vendor 5 to 6 godawn full of text book but not sale to direct us. they told not in stocks and now not coming, but they came to school and sale full stock. why ? pl. look in this matter."
9,"tw: pedophilia, sexual harassment, grooming\n\nsome students at bacoor national high school molino main have come forward with their experiences with some of the teachers there. please spread!!!!\n\nfacebook kept on deleting the posts so we decided to spread awareness here"


22/12/04 16:06:36 WARN org.apache.spark.deploy.yarn.YarnAllocator: Container from a bad node: container_1670165979223_0002_01_000034 on host: hub-msca-bdp-dphub-students-chw387-sw-csh6.c.msca-bdp-students.internal. Exit status: 143. Diagnostics: [2022-12-04 16:06:36.798]Container killed on request. Exit code is 143
[2022-12-04 16:06:36.799]Container exited with a non-zero exit code 143. 
[2022-12-04 16:06:36.799]Killed by external signal
.
22/12/04 16:06:36 WARN org.apache.spark.scheduler.cluster.YarnSchedulerBackend$YarnSchedulerEndpoint: Requesting driver to remove executor 34 for reason Container from a bad node: container_1670165979223_0002_01_000034 on host: hub-msca-bdp-dphub-students-chw387-sw-csh6.c.msca-bdp-students.internal. Exit status: 143. Diagnostics: [2022-12-04 16:06:36.798]Container killed on request. Exit code is 143
[2022-12-04 16:06:36.799]Container exited with a non-zero exit code 143. 
[2022-12-04 16:06:36.799]Killed by external signal
.
22/12/04 16:06:36 ERROR or

In [ ]:
filtered_tweets.write.format("parquet").\
mode('overwrite').\
save('gs://msca-bdp-students-bucket/shared_data/chw387/raw_filtered_tweets2')

## checkpoint

In [14]:
filtered_tweets = spark.read.parquet("gs://msca-bdp-students-bucket/shared_data/chw387/raw_filtered_tweets2")

In [15]:
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")
filtered_tweets2 = (
    filtered_tweets
    .withColumn(
        'created_at',
        from_unixtime(unix_timestamp(
            col("created_at"),
            "EEE MMM dd HH:mm:ss ZZZZ yyyy"
        )).cast(TimestampType()))
    .withColumn("rt_from_user_id",col('retweeted_status').user.id_str)
    .withColumn("location",col('user').location)
    .withColumn("user_desc",col('user').description)
    .withColumn("user_name",col('user')['name'])
    .withColumn("user_follow_count",col('user').followers_count)
    .withColumn("user_id_str",col('user').id_str)
    .withColumn("user_verified",col('user').verified)
    .withColumn("total_tweets",col('user').statuses_count)
    .withColumn("friends_count",col('user').friends_count)
    .withColumn("isRt",~F.col('retweeted_status').isNull())
    .withColumn('place',col('place'))
    .withColumn('tweet_id',col('tweet_id'))
)

In [16]:
filtered_tweets3 = filtered_tweets2.filter(
    (~(
        (F.lower('user_name').contains('sport'))|
        (F.lower('user_desc').contains('sport'))|
        (F.col('user_name')=='hs game update news')
    ))
)

In [17]:
user_rtd_sdf = (
    filtered_tweets3
    .withColumn("rt_from_user_id",col('retweeted_status').user.id_str)
    .groupby('rt_from_user_id').count()
    .withColumnRenamed('count', 'user_rtd_count')
)

In [18]:
filtered_tweets4 = filtered_tweets3.join(
    user_rtd_sdf,
    filtered_tweets2['user'].id_str==user_rtd_sdf['rt_from_user_id'],
    how='left'
).drop('rt_from_user_id','user','retweeted_status')

In [19]:
filtered_tweets5 = filtered_tweets4.withColumnRenamed('user_follow_count', 'f_user_follow_count').join(
    filtered_tweets3.groupby('user_id_str').agg(F.max('user_follow_count').alias('user_follow_count')),
    ['user_id_str'],
    how='left'
).drop('f_user_follow_count')

filtered_tweets6 = filtered_tweets5.withColumnRenamed('total_tweets', 'f_total_tweets').join(
    filtered_tweets3.groupby('user_id_str').agg(F.max('total_tweets').alias('total_tweets')),
    ['user_id_str'],
    how='left'
).drop('f_total_tweets')

filtered_tweets7 = filtered_tweets6.withColumnRenamed('friends_count', 'f_friends_count').join(
    filtered_tweets3.groupby('user_id_str').agg(F.max('friends_count').alias('friends_count')),
    ['user_id_str'],
    how='left'
).drop('f_friends_count')

In [20]:
#msg volume
filtered_tweets8 = filtered_tweets7.join(
    filtered_tweets3.groupby('user_id_str').agg(
        F.sum(
            (~F.col('isRt')).cast(IntegerType())
        ).alias('num_original_tweets'),
        F.count('user_id_str').alias('num_edu_tweets')
    ),
    ['user_id_str'],
    how='left'
).drop('tweet_text')

In [20]:
filtered_tweets.count()

29970964

In [21]:
filtered_tweets8.write.format("parquet").\
mode('overwrite').\
save('gs://msca-bdp-students-bucket/shared_data/chw387/filtered_tweets')

22/12/07 05:15:11 WARN org.apache.spark.deploy.yarn.YarnAllocator: Container from a bad node: container_1670264640795_0033_01_000035 on host: hub-msca-bdp-dphub-students-chw387-sw-hmjc.c.msca-bdp-students.internal. Exit status: 143. Diagnostics: [2022-12-07 05:15:11.223]Container killed on request. Exit code is 143
[2022-12-07 05:15:11.224]Container exited with a non-zero exit code 143. 
[2022-12-07 05:15:11.226]Killed by external signal
.
22/12/07 05:15:11 ERROR org.apache.spark.scheduler.cluster.YarnScheduler: Lost executor 34 on hub-msca-bdp-dphub-students-chw387-sw-hmjc.c.msca-bdp-students.internal: Container from a bad node: container_1670264640795_0033_01_000035 on host: hub-msca-bdp-dphub-students-chw387-sw-hmjc.c.msca-bdp-students.internal. Exit status: 143. Diagnostics: [2022-12-07 05:15:11.223]Container killed on request. Exit code is 143
[2022-12-07 05:15:11.224]Container exited with a non-zero exit code 143. 
[2022-12-07 05:15:11.226]Killed by external signal
.
22/12/07 05:

## save subset


In [22]:
large_dfc = filtered_tweets.limit(2500).cache()

In [23]:
large_dfc.write.format("parquet").\
mode('overwrite').\
save('gs://msca-bdp-students-bucket/shared_data/chw387/largeFinalDf')

## checkpoint

In [24]:
filtered_tweets = spark.read.parquet("gs://msca-bdp-students-bucket/shared_data/chw387/filtered_tweets")

# Explore through different users to find filtering keywords

In [47]:
filtered_tweets.groupby('user_id_str').agg(
    F.first('num_original_tweets').alias('num_original_tweets'),
    F.lower(F.first('user_desc')).alias('user_desc'),
    F.lower(F.first('user_name')).alias('user_name'),
    F.first('num_edu_tweets').alias('num_edu_tweets'),
    F.first('user_follow_count').alias('user_follow_count'),
    F.first('user_rtd_count').alias('user_rtd_count'),
    F.first('total_tweets').alias('total_tweets'),
    F.first('user_verified').alias('user_verified')
).filter(
    #modify here on the fly to examine users
    (F.col('user_name').contains('foundation'))|
    (F.col('user_name').contains('fdn.'))|
    (F.col('user_name').contains('association'))|
    (F.col('user_name').contains('charity'))|
    (F.col('user_name').contains('organization'))|
    (F.col('user_name').contains('united nations'))
)

,user_id_str,num_original_tweets,user_desc,user_name,num_edu_tweets,user_follow_count,user_rtd_count,total_tweets,user_verified
0,21717358,801,school leadership matters.,illinois principals association,866,12620,246.0,51830,False
1,62088756,332,"official account of pmoi/mek, the main iranian opposition group seeking to establish a free, democratic & non-nuclear iran at peace with the world.\n@pmoiran",people's mojahedin organization of iran (pmoi/mek),353,22270,13458.0,140000,True
2,1488136894428889090,250,spotlighting poor education policy that works against student achievement. ➡️ order chaos in our schools by tl zempel at http://amazon.com,school matters foundation,251,97,57.0,1571,False
3,908005757882388480,103,"to provide hands-on service through school and community-based programs, to enhance quality of life for african american and low-income youth and families.",african american youth harvest foundation,178,371,78.0,14319,False
4,321424559,98,alberta school councils' association (asca) representing parents on school councils in alberta.,alberta school councils' association (asca) 🇺🇦,98,3332,71.0,3926,False
...,...,...,...,...,...,...,...,...,...
95,81191957,26,we are a nonprofit organization working to expand access to high-quality charter public schools for all ca families.,ca charter schools association 👨🏻‍🎓👩🏾‍🎓,27,8657,68.0,15893,True
96,763820616,26,the mathematical association exists to support and enhance the teaching and learning of mathematics and its applications.,the mathematical association,58,33519,55.0,30069,False
97,195853535,26,we are a public charity founded by educators for educators to improve public education for all students.,the nea foundation,31,6020,15.0,5575,False
98,1259926170432307207,26,founder of the men of color having answers foundation (m.o.c.h.a.) \nauthor -andrew vs goliath: the american civil czar \nvisionary\nliberator \nhttps://t.co/9v01byozs9,themochamission.org (sc nonprofit organization),28,345,1.0,59882,False


# filtering different user categories

In [27]:
gov_users = filtered_tweets.filter(F.col('user_verified')==True).groupby('user_id_str').agg(
    F.first(F.lower('user_desc')).alias('user_desc'),
    F.first(F.lower('user_name')).alias('user_name'),
    F.first('location').alias('location'),
    F.first('user_rtd_count').alias('user_rtd_count'),
    F.first('user_follow_count').alias('user_follow_count'),
    F.first('total_tweets').alias('total_tweets'),
    F.first('friends_count').alias('friends_count'),
    F.first('num_original_tweets').alias('num_original_tweets'),
    F.first('num_edu_tweets').alias('num_edu_tweets'),
    F.first('user_verified').alias('user_verified')
).filter(
    (
        F.col('user_desc').contains('minist')|
        F.col('user_desc').contains('senate')|
        F.col('user_desc').contains('congress')|
        F.col('user_desc').contains('representative')|
        F.col('user_desc').contains('ambassador')|
        F.col('user_desc').contains('democrat')|
        F.col('user_desc').contains('republican')|
        F.col('user_desc').contains('th district')|
        F.col('user_desc').contains('united states')|
        F.col('user_desc').contains('governor')|
        F.col('user_name').contains('department of')|
        F.col('user_name').contains('city of')|
        F.col('user_name').contains('rep.')
    )
).orderBy('num_edu_tweets',ascending=False)

In [28]:
news_users = filtered_tweets.filter(F.col('user_verified')==True).groupby('user_id_str').agg(
    F.first(F.lower('user_desc')).alias('user_desc'),
    F.first(F.lower('user_name')).alias('user_name'),
    F.first('location').alias('location'),
    F.first('user_rtd_count').alias('user_rtd_count'),
    F.first('user_follow_count').alias('user_follow_count'),
    F.first('total_tweets').alias('total_tweets'),
    F.first('friends_count').alias('friends_count'),
    F.first('num_original_tweets').alias('num_original_tweets'),
    F.first('num_edu_tweets').alias('num_edu_tweets'),
    F.first('user_verified').alias('user_verified')
).filter(
    (
        F.col('user_desc').contains('report')|
        F.col('user_desc').contains('journalist')|
        F.col('user_desc').contains('news')|
        F.col('user_desc').contains('correspondent')|
        F.col('user_desc').contains('editor')|
        F.col('user_desc').contains('anchor')|
        F.col('user_desc').contains('columnist')|
        F.col('user_desc').contains('print media')|
        F.col('user_name').contains('journal')|
        F.col('user_name').contains('press')
    )
).orderBy('num_edu_tweets',ascending=False)

In [29]:
school_users = filtered_tweets.groupby('user_id_str').agg(
    F.first(F.lower('user_desc')).alias('user_desc'),
    F.first(F.lower('user_name')).alias('user_name'),
    F.first('location').alias('location'),
    F.first('user_rtd_count').alias('user_rtd_count'),
    F.first('user_follow_count').alias('user_follow_count'),
    F.first('total_tweets').alias('total_tweets'),
    F.first('friends_count').alias('friends_count'),
    F.first('num_original_tweets').alias('num_original_tweets'),
    F.first('num_edu_tweets').alias('num_edu_tweets'),
    F.first('user_verified').alias('user_verified')
).filter(
    (
        ((F.col('user_verified')==True) & (
                F.col('user_name').contains('school')|
                F.col('user_name').contains('college')|
                F.col('user_name').contains('library')|
                F.col('user_name').contains('university')|
                F.col('user_name').contains('academy')
        )) | (
            F.col('user_desc').contains('principal')|
            F.col('user_desc').contains('teacher')|
            F.col('user_desc').contains('librarian')|
            F.col('user_desc').contains('professor')|
            F.col('user_desc').contains('adjunct scholar')
        )
    )
).orderBy('num_edu_tweets',ascending=False)


In [30]:
ngo_users = filtered_tweets.groupby('user_id_str').agg(
    F.first(F.lower('user_desc')).alias('user_desc'),
    F.first(F.lower('user_name')).alias('user_name'),
    F.first('location').alias('location'),
    F.first('user_rtd_count').alias('user_rtd_count'),
    F.first('user_follow_count').alias('user_follow_count'),
    F.first('total_tweets').alias('total_tweets'),
    F.first('friends_count').alias('friends_count'),
    F.first('num_original_tweets').alias('num_original_tweets'),
    F.first('num_edu_tweets').alias('num_edu_tweets'),
    F.first('user_verified').alias('user_verified')
).filter(
    (F.col('user_name').contains('foundation'))|
    (F.col('user_name').contains('fdn.'))|
    (F.col('user_name').contains('association'))|
    (F.col('user_name').contains('charity'))|
    (F.col('user_name').contains('organization'))|
    (F.col('user_name').contains('united nations'))
)

### Identifying influencers

In [31]:
categorized_user_ids = (
    gov_users[['user_id_str']]
    .union(news_users[['user_id_str']])
    .union(school_users[['user_id_str']])[['user_id_str']]
    .union(ngo_users[['user_id_str']])[['user_id_str']]
    .distinct()
)

In [32]:
uncat_user_tweets = filtered_tweets.join(
    categorized_user_ids,
    'user_id_str',
    'leftanti'
)

In [33]:
influ_users = (
    uncat_user_tweets
    .withColumn('follow_friend_ratio', F.col('user_follow_count')/F.col('friends_count'))
    .groupby('user_id_str').agg(
        F.first('follow_friend_ratio').alias('follow_friend_ratio'),
        F.first(F.lower('user_desc')).alias('user_desc'),
        F.first(F.lower('user_name')).alias('user_name'),
        F.first('location').alias('location'),
        F.first('user_rtd_count').alias('user_rtd_count'),
        F.first('user_follow_count').alias('user_follow_count'),
        F.first('total_tweets').alias('total_tweets'),
        F.first('friends_count').alias('friends_count'),
        F.first('num_original_tweets').alias('num_original_tweets'),
        F.first('num_edu_tweets').alias('num_edu_tweets'),
        F.first('user_verified').alias('user_verified')
    )
    .filter(
        (F.col("follow_friend_ratio")>=2)&
        (F.col("user_follow_count")>=10000)
    ).orderBy('num_edu_tweets',ascending=False)
)

In [34]:
gov_users.write.format("parquet").\
mode('overwrite').\
save('gs://msca-bdp-students-bucket/shared_data/chw387/gov_users')

In [35]:
news_users.write.format("parquet").\
mode('overwrite').\
save('gs://msca-bdp-students-bucket/shared_data/chw387/news_users')

In [36]:
school_users.write.format("parquet").\
mode('overwrite').\
save('gs://msca-bdp-students-bucket/shared_data/chw387/school_users')

22/12/07 05:35:24 WARN org.apache.spark.deploy.yarn.YarnAllocator: Container from a bad node: container_1670264640795_0033_01_000070 on host: hub-msca-bdp-dphub-students-chw387-sw-zcxx.c.msca-bdp-students.internal. Exit status: 143. Diagnostics: [2022-12-07 05:35:24.132]Container killed on request. Exit code is 143
[2022-12-07 05:35:24.133]Container exited with a non-zero exit code 143. 
[2022-12-07 05:35:24.133]Killed by external signal
.
22/12/07 05:35:24 WARN org.apache.spark.scheduler.cluster.YarnSchedulerBackend$YarnSchedulerEndpoint: Requesting driver to remove executor 69 for reason Container from a bad node: container_1670264640795_0033_01_000070 on host: hub-msca-bdp-dphub-students-chw387-sw-zcxx.c.msca-bdp-students.internal. Exit status: 143. Diagnostics: [2022-12-07 05:35:24.132]Container killed on request. Exit code is 143
[2022-12-07 05:35:24.133]Container exited with a non-zero exit code 143. 
[2022-12-07 05:35:24.133]Killed by external signal
.
22/12/07 05:35:24 WARN org

In [37]:
ngo_users.write.format("parquet").\
mode('overwrite').\
save('gs://msca-bdp-students-bucket/shared_data/chw387/ngo_users')

In [38]:
influ_users.write.format("parquet").\
mode('overwrite').\
save('gs://msca-bdp-students-bucket/shared_data/chw387/influ_users')

In [39]:
filtered_tweets = filtered_tweets.join(
    gov_users.select(
        F.col('user_id_str'),
        F.col('user_name').alias('isGov')
    ),
    on='user_id_str',
    how='left'
).withColumn(
    "isGov", F.when(F.col("isGov").isNotNull(), True).otherwise(False)
).join(
    news_users.select(
        F.col('user_id_str'),
        F.col('user_name').alias('isNews')
    ),
    on='user_id_str',
    how='left'
).withColumn(
    "isNews", F.when(F.col("isNews").isNotNull(), True).otherwise(False)
).join(
    school_users.select(
        F.col('user_id_str'),
        F.col('user_name').alias('isSchool')
    ),
    on='user_id_str',
    how='left'
).withColumn(
    "isSchool", F.when(F.col("isSchool").isNotNull(), True).otherwise(False)
).join(
    ngo_users.select(
        F.col('user_id_str'),
        F.col('user_name').alias('isNgo')
    ),
    on='user_id_str',
    how='left'
).withColumn(
    "isNgo", F.when(F.col("isNgo").isNotNull(), True).otherwise(False)
).join(
    influ_users.select(
        F.col('user_id_str'),
        F.col('user_name').alias('isInflu')
    ),
    on='user_id_str',
    how='left'
).withColumn(
    "isInflu", F.when(F.col("isInflu").isNotNull(), True).otherwise(False)
)

In [40]:
filtered_tweets.write.format("parquet").\
mode('overwrite').\
save('gs://msca-bdp-students-bucket/shared_data/chw387/filtered_tweets2')

# Explore through different verified users

In [72]:
filtered_tweets.filter(F.col('user_verified')).groupby('user_id_str').agg(
    F.first('num_original_tweets').alias('num_original_tweets'),
    F.first('user_desc').alias('user_desc'),
    F.first('user_name').alias('user_name'),
    F.first('num_edu_tweets').alias('num_edu_tweets'),
    F.first('user_follow_count').alias('user_follow_count'),
    F.first('user_rtd_count').alias('user_rtd_count'),
    F.first('total_tweets').alias('total_tweets')
).orderBy(
    ['num_edu_tweets','user_rtd_count'],
    ascending=False
).limit(250).toPandas()

,user_id_str,num_original_tweets,user_desc,user_name,num_edu_tweets,user_follow_count,user_rtd_count,total_tweets
0,1276140426,296,"National Director of Research, @SchoolChoiceNow; Adjunct Scholar, @CatoInstitute; Senior Fellow, @ReasonFdn; Exec. Director, @EF_Institute; Forbes 30 Under 30",Corey A. DeAngelis,1746,115445,29255.0,120872
1,147561402,834,"Kenya Power Social Media Support Team. \r\nHere to help with all your power related queries,24/7.",Kenya Power,835,1277674,15.0,2005746
2,19616718,723,Education news and rankings from U.S. News & World Report.,U.S. News Education,726,332588,681.0,83549
3,19675956,671,"We make Gizmos, Reflex, Frax, and Science4Us: innovative, seriously fun math and science online solutions for K-12 classrooms.",ExploreLearning,675,9478,205.0,68371
4,36327407,405,One of India's largest media companies. Latest news from around the world. Retweets are not endorsements,Hindustan Times,554,8665506,1521.0,1150804
5,1383920448,352,"Iran News Update (INU) features original news, analysis, and commentary on Iran events.\n#IranProtests\n#IranRevolution\n\n🔴DM is open to echoing protesters' voice",Iran News Update,522,21506,2587.0,54289
6,23602600,408,"Global HE news, data, insights and events. Home of @THEWorldUniRank, @THEWorldSummits, best practice at THE Campus, and @THEuniadvice for students.",Times Higher Education,512,317641,858.0,156023
7,78874095,372,"Award-winning journalism for schools, including news, analysis, research reviews and teaching and learning guides.",Tes,477,362366,1331.0,118412
8,245424983,214,"AFT Prez. Teacher, Unionist. Fighting 4 children & communities, democracy & justice, public ed & healthcare as a right. Wife of @Skleinbaum. Pronouns: She/her",Randi Weingarten ☮️🇺🇦,448,120020,841.0,201406
9,199399909,83,"As a kid, I wanted to be a superhero, psychologist, philanthropist, philosopher, actor and comedian... So I became a teacher. 🍎",Nicholas Ferroni,446,126152,7724.0,121009


# Quantiles

In [125]:
# Follower counts
(
    filtered_tweets.filter(
        F.col('user_verified')==True
    ).approxQuantile("user_follow_count", [0.01,0.25,0.5,0.75,0.99], 0.01) 
)

[0.0, 7283.0, 23157.0, 89393.0, 303491.0, 132048115.0]

In [99]:
# Retweet counts
(
    filtered_tweets
    .approxQuantile("user_rtd_count", [0.01,0.25,0.5,0.75,0.9,0.99], 0.01) 
)

[1.0, 2.0, 5.0, 24.0, 119.0, 231984.0]

In [100]:
(
    filtered_tweets
    .withColumn('follow_friend_ratio', F.col('user_follow_count')/F.col('friends_count'))
    .approxQuantile("follow_friend_ratio", [0.01,0.25,0.5,0.75,0.9,0.99], 0.01) 
)

[0.0,
 0.3046800382043935,
 0.7019774011299436,
 1.2752634841909485,
 3.4909090909090907,
 13685197.0]

In [ ]:
filtered_tweets

In [107]:
(
    filtered_tweets
    .withColumn('follow_friend_ratio', F.col('user_follow_count')/F.col('friends_count'))
    .groupby('user_id_str').agg(
        F.first('follow_friend_ratio').alias('follow_friend_ratio'),
        F.first('user_desc'),
        F.first('user_name'),
        F.first('user_verified'),
        F.first('user_rtd_count'),
        F.first('num_original_tweets'),
        F.first('num_edu_tweets').alias('num_edu_tweets'),
        F.first('user_follow_count').alias('user_follow_count'),
        F.first('total_tweets')
    )
    .filter(
        (F.col("follow_friend_ratio")>=4.0)&
        (F.col("user_follow_count")>=10000)
    ).orderBy('num_edu_tweets',ascending=False)
).limit(200).toPandas()

,user_id_str,follow_friend_ratio,first(user_desc),first(user_name),first(user_verified),first(user_rtd_count),first(num_original_tweets),num_edu_tweets,user_follow_count,first(total_tweets)
0,1276140426,7.496429e+01,"National Director of Research, @SchoolChoiceNow; Adjunct Scholar, @CatoInstitute; Senior Fellow, @ReasonFdn; Exec. Director, @EF_Institute; Forbes 30 Under 30",Corey A. DeAngelis,True,29255.0,296,1746,115445,120872
1,1387798747329961989,4.460752e+01,Covering Texas High School athletics for the http://Rivals.com network. Visit us today at http://Lonestarpreps.com.,LoneStarPreps.com,False,3472.0,1202,1202,21367,7705
2,1387867666614943748,4.451311e+01,Providing coverage of the top high school athletes across the entire state. http://GaVarsity.com is a proud member of http://Rivals.com.,GaVarsity,False,1392.0,944,944,11885,5871
3,321003806,3.971842e+02,We are offering you the best Global Research Positions in Agriculture and Biosciences. Follow us and take the next step in your academic career,Agristok,False,10236.0,453,930,15093,9016
4,47095078,1.479760e+01,Editor of http://IowaPreps.com,Iowa Preps,False,1217.0,889,889,22226,55609
5,21717358,1.302374e+01,School leadership matters.,Illinois Principals Association,False,302.0,801,866,12620,51830
6,147561402,6.492906e+01,"Kenya Power Social Media Support Team. \r\nHere to help with all your power related queries,24/7.",Kenya Power,True,15.0,834,835,1277674,2005746
7,3517689082,4.746781e+00,"Sports journalist, with more than 20 years' experience covering high schools, from Kentucky.\n\nAccepts ALL DMs (even non followers)",Nate Bryan,False,733.0,643,834,14378,88677
8,212922754,4.948598e+01,"I tweet and retweet about Studying Abroad, Schools and Scholarships. STEM-Grad Assistantships tips.",Felo Anifel,False,14792.0,529,792,37065,22189
9,19616718,4.710878e+02,Education news and rankings from U.S. News & World Report.,U.S. News Education,True,681.0,723,726,332588,83549
